In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from requests.exceptions import HTTPError
from random import randint
import time

# first download the list of words

In [2]:
def retrieve_webpage(req_url):
    """Get a webpage with the requests module & return the response"""
    user_agents = [
        {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"},
        {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36"},
        {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0"},
        {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36"},
        {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"},
        {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36"},
        {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"},
        {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.2 Safari/605.1.15"},
        {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15"},
        {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:91.0) Gecko/20100101 Firefox/91.0"},
        {'User-Agent': "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36"},
        {'User-Agent': "Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36"},
        {'User-Agent': "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:15.0) Gecko/20100101 Firefox/15.0.1"},
        {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
    ]
    try:
        req_response = requests.get(req_url, headers=user_agents[randint(0, len(user_agents))])
        req_response.raise_for_status()
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err} for URL {req_url}')
        return None
    except Exception as err:
        print(f'Other error occurred: {err} for URL {req_url}')
        return None
    else:
        # print('Request success!')
        return req_response

In [3]:
time.sleep(2.4) # in sec
url_wordlist = "https://www.oxfordlearnersdictionaries.com/wordlists/oxford3000-5000"
req_response = retrieve_webpage(url_wordlist)

if not req_response :
    print("fail")
elif str(req_response) == '<Response [200]>':
    print("success")
else:
    print("something wrong")

success


In [4]:
s = BeautifulSoup(req_response.content, 'html.parser')
all_elts = s.find_all("li")[34:-44]
words, levels, def_urls, types, sound_urls = [], [], [], [], []

for i in all_elts:
    i = str(i)
    if "data-hw" in i:
        words.append(i.split('data-hw="')[1].split('"')[0])
    else:
        words.append("No word")
    if "data-ox5000" in i:
        levels.append(i.split('data-ox5000="')[1].split('"')[0])
    else:
        levels.append("No level")
    if "href" in i:
        def_urls.append(i.split('href="')[1].split('"')[0])
    else:
        def_urls.append("No def_url")
    if 'class="pos' in i:
        types.append(i.split('class="pos">')[1].split('<')[0])
    else:
        types.append("No types")
    if "data-src-mp3" in i:
        sound_urls.append(i.split('pron-us" data-src-mp3="')[1].split('"')[0])
    else:
        sound_urls.append("No sound url")
    

len(words), len(levels), len(def_urls), len(types), len(sound_urls)

(5948, 5948, 5948, 5948, 5948)

In [5]:
df = pd.DataFrame(list(zip(words, levels, def_urls, types, sound_urls)), columns =['words', 'levels', 'def_urls', 'types', 'sound_urls'])
df.to_csv('words_list', index=False)
df.head(3)

,words,levels,def_urls,types,sound_urls
0,a,a1,/definition/english/a_1,indefinite article,/media/english/us_pron/a/a__/a__us/a__us_2_rr.mp3
1,abandon,b2,/definition/english/abandon_1,verb,/media/english/us_pron/a/aba/aband/abandon__us...
2,ability,a2,/definition/english/ability_1,noun,/media/english/us_pron/a/abi/abili/ability__us...


In [6]:
df.tail(3)

,words,levels,def_urls,types,sound_urls
5945,youth,b1,/definition/english/youth,noun,/media/english/us_pron/y/you/youth/youth__us_1...
5946,zero,a2,/definition/english/zero_1,number,/media/english/us_pron/z/zer/zero_/zero__us_1_...
5947,zone,b2,/definition/english/zone_1,noun,/media/english/us_pron/z/zon/zone_/zone__us_1.mp3


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5948 entries, 0 to 5947
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   words       5948 non-null   object
 1   levels      5948 non-null   object
 2   def_urls    5948 non-null   object
 3   types       5948 non-null   object
 4   sound_urls  5948 non-null   object
dtypes: object(5)
memory usage: 232.5+ KB


In [8]:
df[df['levels'] == 'No level'].shape[0]

5

In [9]:
df[df['levels'] == 'No level']

,words,levels,def_urls,types,sound_urls
47,accounting,No level,/definition/english/accounting,noun,/media/english/us_pron/a/acc/accou/accounting_...
233,angrily,No level,/definition/english/angrily,adverb,/media/english/us_pron/a/ang/angri/angrily__us...
889,cleaning,No level,/definition/english/cleaning,noun,/media/english/us_pron/c/cle/clean/cleaning__u...
2058,feeding,No level,/definition/english/feeding,noun,/media/english/us_pron/f/fee/feedi/feeding__us...
3176,major,No level,/definition/english/major_2,noun,/media/english/us_pron/m/maj/major/major__us_2...


In [10]:
df['levels'].unique()

array(['a1', 'b2', 'a2', 'c1', 'b1', 'No level'], dtype=object)

In [11]:
df.loc[df[df['levels'] == 'No level'].index, 'levels'] = 'a1'

In [12]:
df[df['sound_urls'] == 'No sound url'].shape[0]

1

In [13]:
df[df['sound_urls'] == 'No sound url']

,words,levels,def_urls,types,sound_urls
3559,nursing,b2,/definition/english/nursing,noun,No sound url


In [14]:
df[df.duplicated()].shape[0]

0

# Let's clean & transform the data a little bit

In [15]:
df.head()

,words,levels,def_urls,types,sound_urls
0,a,a1,/definition/english/a_1,indefinite article,/media/english/us_pron/a/a__/a__us/a__us_2_rr.mp3
1,abandon,b2,/definition/english/abandon_1,verb,/media/english/us_pron/a/aba/aband/abandon__us...
2,ability,a2,/definition/english/ability_1,noun,/media/english/us_pron/a/abi/abili/ability__us...
3,able,a2,/definition/english/able_1,adjective,/media/english/us_pron/a/abl/able_/able__us_2.mp3
4,abolish,c1,/definition/english/abolish,verb,/media/english/us_pron/a/abo/aboli/abolish__us...


In [16]:
BASE_URL = "https://www.oxfordlearnersdictionaries.com"

df['def_urls'] = df['def_urls'].apply(lambda x: BASE_URL + x)
df['levels'] = df['levels'].apply(lambda x: x.upper())
df['types'] = df['types'].apply(lambda x: "(" + x + ")")
df['sound_urls'] = df['sound_urls'].apply(lambda x: BASE_URL + x)
df['sound_files'] = df['sound_urls'].apply(lambda x: x.split("/")[-1])
df.head()

,words,levels,def_urls,types,sound_urls,sound_files
0,a,A1,https://www.oxfordlearnersdictionaries.com/def...,(indefinite article),https://www.oxfordlearnersdictionaries.com/med...,a__us_2_rr.mp3
1,abandon,B2,https://www.oxfordlearnersdictionaries.com/def...,(verb),https://www.oxfordlearnersdictionaries.com/med...,abandon__us_2.mp3
2,ability,A2,https://www.oxfordlearnersdictionaries.com/def...,(noun),https://www.oxfordlearnersdictionaries.com/med...,ability__us_4.mp3
3,able,A2,https://www.oxfordlearnersdictionaries.com/def...,(adjective),https://www.oxfordlearnersdictionaries.com/med...,able__us_2.mp3
4,abolish,C1,https://www.oxfordlearnersdictionaries.com/def...,(verb),https://www.oxfordlearnersdictionaries.com/med...,abolish__us_1.mp3


# Retrieve the sound files

In [ ]:
!mkdir sounds

In [ ]:
with open("./sounds" + "/" + "url_list.txt", "w") as outfile:
    outfile.write("\n".join(set(df['sound_urls'])))

In [ ]:
!cd sounds, wget -i url_list.txt

In [ ]:
from os import listdir
from os.path import isfile, join

files_list = [f for f in listdir("./sounds") if isfile(join("./sounds", f))]
print(len(files_list))
print(files_list[:10])

In [ ]:
df.to_csv('word_list.csv', index=False)

# Final step scraping of all the pages

In [119]:
def get_data(resp):
    """Parse HTTP response of a single webpage with BS4 and return relevant data"""
    soup = BeautifulSoup(resp.content, 'html.parser')
    try:
        phonetic = soup.find_all("div", class_="phons_n_am")[0].find_all("span", class_="phon")[0].contents[0]
    except:
        pass
    try:
        senses = soup.find_all("li", class_="sense")
    except:
        pass
    try:
        definitions = [se.find_all(class_="def")[0].contents[0] for se in senses]
        definitions = [f"{i+1}. {def_.replace(';', ',')}" for i, def_ in enumerate(definitions)]
    except:
        pass
    
    try:
        examples = [] # a list of (list of examples for one definition)
        for se in senses:
            all_examples = se.find_all("ul", class_="examples")[0].find_all(htag="li")
            try:
                all_examples = [e.contents[0].text for e in all_examples]
            except:
                try:
                    all_examples = [ex.find_all(class_="x")[0].text for ex in all_ex]
                except:
                    pass
            examples.append("".join(["<dd>- " + e.replace(';', ',') + "<br>" for e in all_examples ]))
    except:
        pass
    return phonetic, definitions, examples

df_test = df.iloc[:20]
df_test

,words,levels,def_urls,types,sound_urls,sound_files
0,a,A1,https://www.oxfordlearnersdictionaries.com/def...,(indefinite article),https://www.oxfordlearnersdictionaries.com/med...,a__us_2_rr.mp3
1,abandon,B2,https://www.oxfordlearnersdictionaries.com/def...,(verb),https://www.oxfordlearnersdictionaries.com/med...,abandon__us_2.mp3
2,ability,A2,https://www.oxfordlearnersdictionaries.com/def...,(noun),https://www.oxfordlearnersdictionaries.com/med...,ability__us_4.mp3
3,able,A2,https://www.oxfordlearnersdictionaries.com/def...,(adjective),https://www.oxfordlearnersdictionaries.com/med...,able__us_2.mp3
4,abolish,C1,https://www.oxfordlearnersdictionaries.com/def...,(verb),https://www.oxfordlearnersdictionaries.com/med...,abolish__us_1.mp3
5,abortion,C1,https://www.oxfordlearnersdictionaries.com/def...,(noun),https://www.oxfordlearnersdictionaries.com/med...,abortion__us_1.mp3
6,about,A1,https://www.oxfordlearnersdictionaries.com/def...,(adverb),https://www.oxfordlearnersdictionaries.com/med...,about__us_1.mp3
7,about,A1,https://www.oxfordlearnersdictionaries.com/def...,(preposition),https://www.oxfordlearnersdictionaries.com/med...,about__us_1.mp3
8,above,A1,https://www.oxfordlearnersdictionaries.com/def...,(adverb),https://www.oxfordlearnersdictionaries.com/med...,above__us_2.mp3
9,above,A1,https://www.oxfordlearnersdictionaries.com/def...,(preposition),https://www.oxfordlearnersdictionaries.com/med...,above__us_2.mp3


In [120]:
for url in df_test["def_urls"].values: #list(df_test["def_urls"]):
    time.sleep(randint(2, 9)) # in sec
    req_response = retrieve_webpage(url.strip())
    if not req_response :
        continue
    elif str(req_response) == '<Response [200]>':
        # success"
        phonetic, definitions, examples = get_data(req_response)
        df_test.loc[df_test[df_test['def_urls'] == url].index, 'phonetic'] = phonetic
        for i, def_ in enumerate(definitions):
            #df_test.loc[df_test['def_urls'] == url][f'definition_{i+1}'] = def_
            df_test.loc[df_test[df_test['def_urls'] == url].index, f'definition_{i+1}'] = def_
        for i, ex in enumerate(examples):
            df_test.loc[df_test[df_test['def_urls'] == url].index, f'examples_{i+1}'] = ex
    else:
        print("something wrong !! for URL {url}")
        
df_test.head()

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

Other error occurred: list index out of range for URL https://www.oxfordlearnersdictionaries.com/definition/english/abuse_1


,words,levels,def_urls,types,sound_urls,sound_files,phonetic,definition_1,definition_2,definition_3,...,examples_3,examples_4,examples_5,examples_6,examples_7,examples_8,examples_9,examples_10,definition_11,definition_12
0,a,A1,https://www.oxfordlearnersdictionaries.com/def...,(indefinite article),https://www.oxfordlearnersdictionaries.com/med...,a__us_2_rr.mp3,/ə/,1. used before countable or singular nouns ref...,2. used to show that somebody/something is a m...,"3. any, every",...,<dd>- A lion is a dangerous animal.<br>,<dd>- a good knowledge of French<br><dd>- a sa...,<dd>- a knife and fork<br>,<dd>- A thousand people were there.<br>,<dd>- They cost 50p a kilo.<br><dd>- I can typ...,<dd>- She's a little Greta Thunberg.<br>,<dd>- There's a Mrs Green to see you.<br>,<dd>- She died on a Tuesday.<br>,NaN,NaN
1,abandon,B2,https://www.oxfordlearnersdictionaries.com/def...,(verb),https://www.oxfordlearnersdictionaries.com/med...,abandon__us_2.mp3,/əˈbændən/,"1. to leave somebody, especially somebody you ...","2. to leave a thing or place, especially becau...","3. to stop doing something, especially before ...",...,<dd>- They abandoned the match because of rain...,<dd>- The baby had been abandoned by its mothe...,<dd>- He abandoned himself to despair.<br>,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ability,A2,https://www.oxfordlearnersdictionaries.com/def...,(noun),https://www.oxfordlearnersdictionaries.com/med...,ability__us_4.mp3,/əˈbɪləti/,1. the fact that somebody/something is able to...,2. a level of skill or intelligence,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,able,A2,https://www.oxfordlearnersdictionaries.com/def...,(adjective),https://www.oxfordlearnersdictionaries.com/med...,able__us_2.mp3,/ˈeɪbl/,"1. to have the skill, intelligence, opportunit...","2. intelligent, good at something",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,abolish,C1,https://www.oxfordlearnersdictionaries.com/def...,(verb),https://www.oxfordlearnersdictionaries.com/med...,abolish__us_1.mp3,/əˈbɑːlɪʃ/,"1. to officially end a law, a system or an ins...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [121]:
df_test[[col for col in df_test.columns if "def" in col]]

,def_urls,definition_1,definition_2,definition_3,definition_4,definition_5,definition_6,definition_7,definition_8,definition_9,definition_10,definition_11,definition_12
0,https://www.oxfordlearnersdictionaries.com/def...,1. used before countable or singular nouns ref...,2. used to show that somebody/something is a m...,"3. any, every",4. used before uncountable nouns when these ha...,5. used in front of two nouns that are seen as...,6. used instead of,"7. used when talking about prices, quantities ...",8. a person like somebody,9. used before somebody’s name to show that th...,10. used before the names of days of the week ...,NaN,NaN
1,https://www.oxfordlearnersdictionaries.com/def...,"1. to leave somebody, especially somebody you ...","2. to leave a thing or place, especially becau...","3. to stop doing something, especially before ...","4. to stop supporting or helping somebody, to ...",5. to feel an emotion so strongly that you can...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.oxfordlearnersdictionaries.com/def...,1. the fact that somebody/something is able to...,2. a level of skill or intelligence,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.oxfordlearnersdictionaries.com/def...,"1. to have the skill, intelligence, opportunit...","2. intelligent, good at something",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.oxfordlearnersdictionaries.com/def...,"1. to officially end a law, a system or an ins...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,https://www.oxfordlearnersdictionaries.com/def...,1. the deliberate ending of a,2. a medical operation to end a,3. the process of giving birth to a baby befor...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,https://www.oxfordlearnersdictionaries.com/def...,"1. a little more or less than, a little before...","2. nearly, very close to","3. in many directions, here and there","4. in no particular order, in various places",5. doing nothing in particular,6. able to be found in a place,7. facing the opposite direction,"8. almost, very nearly",9. approximately,10. able to go outside again after an illness,11. travelling around a place,12. used to say that you have finished telling...
7,https://www.oxfordlearnersdictionaries.com/def...,"1. on the subject of somebody/something, in co...",2. used to describe the purpose or an aspect o...,"3. busy with something, doing something","4. in many directions in a place, here and there","5. in various parts of a place, here and there","6. next to a place or person, in the area ment...",7. surrounding somebody/something,8. used when asking for information about some...,9. used to make a suggestion,NaN,NaN,NaN
8,https://www.oxfordlearnersdictionaries.com/def...,1. at or to a higher place,"2. greater in number, level or age",3. earlier in something written or printed,4. (to do) even more than is expected or demanded,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,https://www.oxfordlearnersdictionaries.com/def...,1. at or to a higher place or position than so...,"2. more than something, greater in number, lev...",3. of greater importance or of higher quality ...,4. too good or too honest to do something,5. louder or clearer than another sound,"6. most important of all, especially",7. having too high an opinion of yourself,8. in addition to something,NaN,NaN,NaN,NaN


In [122]:
df_test[[col for col in df_test.columns if "exam" in col]]

,examples_1,examples_2,examples_3,examples_4,examples_5,examples_6,examples_7,examples_8,examples_9,examples_10
0,<dd>- a man/horse/unit<br><dd>- an aunt/egg/ho...,<dd>- Their new car's a BMW.<br><dd>- She's a ...,<dd>- A lion is a dangerous animal.<br>,<dd>- a good knowledge of French<br><dd>- a sa...,<dd>- a knife and fork<br>,<dd>- A thousand people were there.<br>,<dd>- They cost 50p a kilo.<br><dd>- I can typ...,<dd>- She's a little Greta Thunberg.<br>,<dd>- There's a Mrs Green to see you.<br>,<dd>- She died on a Tuesday.<br>
1,<dd>- The baby had been abandoned by its mothe...,<dd>- The baby had been abandoned by its mothe...,<dd>- They abandoned the match because of rain...,<dd>- The baby had been abandoned by its mothe...,<dd>- He abandoned himself to despair.<br>,NaN,NaN,NaN,NaN,NaN
2,<dd>- People with the disease may lose their a...,<dd>- Their athletic ability is stunning.<br><...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,<dd>- You must be able to speak French for thi...,<dd>- She's the ablest student in the class.<b...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,<dd>- This tax should be abolished.<br>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,<dd>- to support/oppose abortion<br><dd>- a wo...,<dd>- She decided to have an abortion.<br><dd>...,<dd>- Higher amounts of caffeine may be associ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,<dd>- It costs about $10.<br><dd>- They waited...,<dd>- I'm just about ready.<br><dd>- This is a...,<dd>- The children were rushing about in the g...,<dd>- Her books were lying about on the floor....,<dd>- People were standing about in the road.<br>,<dd>- There was nobody about.<br><dd>- There's...,<dd>- He brought the ship about.<br>,<dd>- I've met just about everyone.<br><dd>- ‘...,<dd>- She should be arriving just about now.<br>,NaN
7,<dd>- a book about flowers<br><dd>- Tell me al...,<dd>- Movies are all about making money these ...,<dd>- Everywhere people were going about their...,<dd>- We wandered about the town for an hour o...,<dd>- The papers were strewn about the room.<br>,<dd>- She's somewhere about the office.<br>,<dd>- She wore a shawl about her shoulders.<br>,<dd>- How about Ruth? Have you heard from her?...,<dd>- The baby had been abandoned by its mothe...,NaN
8,<dd>- Put it on the shelf above.<br><dd>- Seen...,<dd>- increases of 5% and above<br><dd>- A sco...,"<dd>- As was stated above…<br><dd>- See above,...",<dd>- Our staff members will go above and beyo...,NaN,NaN,NaN,NaN,NaN,NaN
9,<dd>- The water came above our knees.<br><dd>-...,<dd>- Inflation is above 6%.<br><dd>- Temperat...,<dd>- I rate her above most other players of h...,<dd>- She's not above lying when it suits her....,<dd>- I couldn't hear her above the noise of t...,"<dd>- Above all, keep in touch.<br>",NaN,NaN,NaN,NaN


In [103]:
df_test[df_test["words"] == "about"]["def_urls"].values

array(['https://www.oxfordlearnersdictionaries.com/definition/english/about_2',
       'https://www.oxfordlearnersdictionaries.com/definition/english/about_1'],
      dtype=object)

In [109]:
list(df_test["def_urls"])

['https://www.oxfordlearnersdictionaries.com/definition/english/a_1',
 'https://www.oxfordlearnersdictionaries.com/definition/english/abandon_1',
 'https://www.oxfordlearnersdictionaries.com/definition/english/ability_1',
 'https://www.oxfordlearnersdictionaries.com/definition/english/able_1',
 'https://www.oxfordlearnersdictionaries.com/definition/english/abolish',
 'https://www.oxfordlearnersdictionaries.com/definition/english/abortion',
 'https://www.oxfordlearnersdictionaries.com/definition/english/about_2',
 'https://www.oxfordlearnersdictionaries.com/definition/english/about_1',
 'https://www.oxfordlearnersdictionaries.com/definition/english/above_2',
 'https://www.oxfordlearnersdictionaries.com/definition/english/above_1',
 'https://www.oxfordlearnersdictionaries.com/definition/english/abroad',
 'https://www.oxfordlearnersdictionaries.com/definition/english/absence',
 'https://www.oxfordlearnersdictionaries.com/definition/english/absent_1',
 'https://www.oxfordlearnersdictionari

In [110]:
df_test["def_urls"].values

array(['https://www.oxfordlearnersdictionaries.com/definition/english/a_1',
       'https://www.oxfordlearnersdictionaries.com/definition/english/abandon_1',
       'https://www.oxfordlearnersdictionaries.com/definition/english/ability_1',
       'https://www.oxfordlearnersdictionaries.com/definition/english/able_1',
       'https://www.oxfordlearnersdictionaries.com/definition/english/abolish',
       'https://www.oxfordlearnersdictionaries.com/definition/english/abortion',
       'https://www.oxfordlearnersdictionaries.com/definition/english/about_2',
       'https://www.oxfordlearnersdictionaries.com/definition/english/about_1',
       'https://www.oxfordlearnersdictionaries.com/definition/english/above_2',
       'https://www.oxfordlearnersdictionaries.com/definition/english/above_1',
       'https://www.oxfordlearnersdictionaries.com/definition/english/abroad',
       'https://www.oxfordlearnersdictionaries.com/definition/english/absence',
       'https://www.oxfordlearnersdiction

In [99]:
url = "https://www.oxfordlearnersdictionaries.com/definition/english/about_1"
req = retrieve_webpage(url)
get_data(req)

('/əˈbaʊt/',
 ['1. on the subject of somebody/something, in connection with somebody/something',
  '2. used to describe the purpose or an aspect of something',
  '3. busy with something, doing something',
  '4. in many directions in a place, here and there',
  '5. in various parts of a place, here and there',
  '6. next to a place or person, in the area mentioned',
  '7. surrounding somebody/something',
  '8. used when asking for information about somebody/something',
  '9. used to make a suggestion'],
 ["<dd>- a book about flowers<br><dd>- Tell me all about it.<br><dd>- What's she so angry about?<br><dd>- There's something strange about him.<br><dd>- There's something about this business that is not quite right.<br><dd>- I don't know what you're on about (= talking about).<br><dd>- There's nothing you can do about it now.<br>",
  '<dd>- Movies are all about making money these days.<br><dd>- What was all that about? (= what was the reason for what has just happened?)<br>',
  '<dd>- Eve

In [38]:
ex

[]

In [39]:
s = BeautifulSoup(req.content, 'html.parser')
senses = s.find_all("li", class_="sense")
senses

[<li cefr="b2" class="sense" hclass="sense" htag="li" id="abandon_sng_1" ox3000="y" sensenum="1"><span class="sensetop" hclass="sensetop" htag="span"><div class="symbols" hclass="symbols" htag="div"><a href="https://www.oxfordlearnersdictionaries.com/wordlists/oxford3000-5000?dataset=english&amp;list=ox3000&amp;level=b2"><span class="ox3ksym_b2"> </span></a></div></span> <span class="def" hclass="def" htag="span">to leave somebody, especially somebody you are responsible for, with no intention of returning</span><ul class="examples" hclass="examples" htag="ul"><li class="" htag="li"> <span class="cf" hclass="cf" htag="span">abandon somebody</span> <span class="x">The baby had been abandoned by its mother.</span></li><li class="" htag="li"><span class="x">People often simply abandon their pets when they go abroad.</span></li><li class="" htag="li"> <span class="cf" hclass="cf" htag="span">abandon somebody to something</span> <span class="x">‘We have been <span class="cl">abandoned to ou

In [40]:
len(senses)

5

In [ ]:
        examples = [] # a list of (list of examples for one definition)
        for se in senses:
            all_examples = se.find_all("ul", class_="examples")[0].find_all(htag="li")
            try:
                all_examples = [e.contents[0].text for e in all_examples]
                examples.append("".join(["<dd>- " + e.replace(';', ',') + "<br>" for e in all_examples ]))

In [81]:
all_ex = senses[0].find_all("ul", class_="examples")[0].find_all(htag="li")
all_ex

[<li class="" htag="li"> <span class="cf" hclass="cf" htag="span">abandon somebody</span> <span class="x">The baby had been abandoned by its mother.</span></li>,
 <li class="" htag="li"><span class="x">People often simply abandon their pets when they go abroad.</span></li>,
 <li class="" htag="li"> <span class="cf" hclass="cf" htag="span">abandon somebody to something</span> <span class="x">‘We have been <span class="cl">abandoned to our fate</span>,’ said one resident.</span></li>,
 <li class="" htag="li"><span class="x">The study showed a deep fear among the elderly of being abandoned to the care of strangers.</span></li>]

In [82]:
[e.contents[0].txt for e in all_ex]

AttributeError: 'NavigableString' object has no attribute 'txt'

In [83]:
type(all_ex[0].contents[0])

bs4.element.NavigableString

In [84]:
[e.contents for e in all_ex]

[[' ',
  <span class="cf" hclass="cf" htag="span">abandon somebody</span>,
  ' ',
  <span class="x">The baby had been abandoned by its mother.</span>],
 [<span class="x">People often simply abandon their pets when they go abroad.</span>],
 [' ',
  <span class="cf" hclass="cf" htag="span">abandon somebody to something</span>,
  ' ',
  <span class="x">‘We have been <span class="cl">abandoned to our fate</span>,’ said one resident.</span>],
 [<span class="x">The study showed a deep fear among the elderly of being abandoned to the care of strangers.</span>]]

In [85]:
all_ex

[<li class="" htag="li"> <span class="cf" hclass="cf" htag="span">abandon somebody</span> <span class="x">The baby had been abandoned by its mother.</span></li>,
 <li class="" htag="li"><span class="x">People often simply abandon their pets when they go abroad.</span></li>,
 <li class="" htag="li"> <span class="cf" hclass="cf" htag="span">abandon somebody to something</span> <span class="x">‘We have been <span class="cl">abandoned to our fate</span>,’ said one resident.</span></li>,
 <li class="" htag="li"><span class="x">The study showed a deep fear among the elderly of being abandoned to the care of strangers.</span></li>]

In [86]:
all_ex[0].find_all(class_="x")[0].text

'The baby had been abandoned by its mother.'

In [88]:
[ex.find_all(class_="x")[0].text for ex in all_ex]

['The baby had been abandoned by its mother.',
 'People often simply abandon their pets when they go abroad.',
 '‘We have been abandoned to our fate,’ said one resident.',
 'The study showed a deep fear among the elderly of being abandoned to the care of strangers.']

In [41]:
examples = [] # a list of (list of examples for one definition)
for se in senses:
    all_examples = se.find_all("ul", class_="examples")[0].find_all(htag="li")
    all_examples = [e.contents[0].text for e in all_examples]
    examples.append("".join(["<dd>- " + e.replace(';', ',') + "<br>" for e in all_examples ]))

AttributeError: 'NavigableString' object has no attribute 'text'

In [42]:
examples

[]

In [26]:
ss = s.find_all("li", class_="sense")

In [ ]:
examples[0]

In [ ]:
"".join(["<dd>- " + e.replace(';', ',') + "<br>" for e in examples[0] ])

In [ ]:
final_examples = []
for ex in examples:
    temp_ = []
    for e in ex:
        temp_.append("<dd>- " + e.replace(';', ',') + "<br>")
    final_examples.append("".join(temp_))

In [ ]:
[ex for ex in examples]

for ex in examples:
    temp_ = []
    for e in ex:
        temp_.append("<dd>- " + e.replace(';', ',') + "<br>")
    final_examples.append("".join(temp_))

In [ ]:
["".join("<dd>- " + e.replace(';', ',') + "<br>") for e in ex for ex in examples]

In [ ]:
["".join("<dd>- " + e.replace(';', ',') + "<br>") for ex in examples for e in ex ]

In [ ]:
del final_examples

In [ ]:
            all_examples = se.find_all("ul", class_="examples")[0].find_all(htag="li")
            examples.append([e.contents[0].text for e in all_examples])
            examples = ["".join("<dd>- " + e.replace(';', ',') + "<br>") for ex in examples for e in ex ]

In [ ]:
        {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:53.0) Gecko/20100101 Firefox/53.0"},
        {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:53.0) Gecko/20100101 Firefox/53.0"},
        {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:53.0) Gecko/20100101 Firefox/53.0"},
        {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:53.0) Gecko/20100101 Firefox/53.0"}